In [ ]:
import numpy as np
import nibabel as nib
from nilearn import plotting
from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.image import coord_transform, load_img

In [ ]:
def turn_MNI_to_regions(mni_coords):
    atlas_data = fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7, resolution_mm=1)
    parcellation_img = load_img(atlas_data.maps)
    parcellation_data = parcellation_img.get_fdata()
    voxel_coords = [
        coord_transform(mni[0], mni[1], mni[2], np.linalg.inv(parcellation_img.affine))
        for mni in mni_coords
    ]

    for mni, voxel in zip(mni_coords, voxel_coords):
        x, y, z = np.round(voxel).astype(int)
        label_index = parcellation_data[x, y, z]
        label_name = atlas_data.labels[int(label_index) - 1] if label_index > 0 else 'No label'
        print(f"MNI coordinates {mni} overlap with label: {label_name}")
    

In [ ]:
import numpy as np
from nilearn.datasets import fetch_atlas_schaefer_2018
from nilearn.image import coord_transform, load_img

def turn_MNI_to_regions_and_neighbors_2(mni_coords, neighborhood_radius=8):
    atlas_data = fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7, resolution_mm=1)
    parcellation_img = load_img(atlas_data.maps)
    parcellation_data = parcellation_img.get_fdata()
    
    voxel_coords = [
        coord_transform(mni[0], mni[1], mni[2], np.linalg.inv(parcellation_img.affine))
        for mni in mni_coords
    ]

    for mni, voxel in zip(mni_coords, voxel_coords):
        x, y, z = np.round(voxel).astype(int)
        if (x < 0 or x >= parcellation_data.shape[0] or
            y < 0 or y >= parcellation_data.shape[1] or
            z < 0 or z >= parcellation_data.shape[2]):
            label_name = 'Invalid voxel coordinates'
            neighbor_names = 'No neighbors'
        else:
            label_index = parcellation_data[x, y, z]
            label_name = atlas_data.labels[int(label_index) - 1] if label_index > 0 else 'No label'
            print(f"MNI coordinates {mni} overlap with label: {label_name}")

            # Check neighboring voxels within the specified radius
            neighbor_labels = set()
            for i in range(-neighborhood_radius, neighborhood_radius + 1):
                for j in range(-neighborhood_radius, neighborhood_radius + 1):
                    for k in range(-neighborhood_radius, neighborhood_radius + 1):
                        nx, ny, nz = x + i, y + j, z + k
                        if (0 <= nx < parcellation_data.shape[0] and
                            0 <= ny < parcellation_data.shape[1] and
                            0 <= nz < parcellation_data.shape[2]):
                            neighbor_label = parcellation_data[nx, ny, nz]
                            if neighbor_label > 0 and neighbor_label != label_index:
                                neighbor_labels.add(int(neighbor_label))
                            
            neighbor_names = [atlas_data.labels[nl - 1] for nl in neighbor_labels]

        print(f"Neighboring regions: {neighbor_names if neighbor_names else 'No neighbors'}")




In [ ]:
import numpy as np
import nibabel as nib
from nilearn import plotting
from nilearn.datasets import fetch_atlas_schaefer_2018

def plot_region(labels_of_interest):

    atlas_data = fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7, resolution_mm=1)
    parcellation_img = atlas_data.maps

    parcellation_map = nib.load(parcellation_img)

    
    composite_mask = np.zeros_like(parcellation_map.get_fdata(), dtype=np.uint8)

    for label in labels_of_interest:
        if label in atlas_data.labels:
            label_index = np.where(atlas_data.labels == label)[0][0] + 1  # Add 1 because labels start from 1 in the parcellation map

            region_mask = np.where(parcellation_map.get_fdata() == label_index, 1, 0).astype(np.uint8)  # Convert to np.uint8

            composite_mask += region_mask

        else:
            print(f"Label '{label}' not found in the atlas.")

    composite_nifti = nib.Nifti1Image(composite_mask, parcellation_map.affine)

    plotting.plot_roi(composite_nifti, title="Composite ROI")